# Ice shelf plumes

This notebook will show the model for buoyant meltwater plumes under ice shelves.
We'll use a synthetic geometry for this notebook; in later examples we'll use real data.
You can think of the plume model as consisting mostly of the shallow water equations with friction, with the sign of gravity flipped, and with a few extra fields along for the ride.

In [ ]:
import firedrake
Lx, Ly = 20e3, 20e3
nx, ny = 20, 20
mesh = firedrake.RectangleMesh(nx, ny, Lx, Ly, diagonal='crossed')

In [ ]:
degree = 1
Q = firedrake.FunctionSpace(mesh, 'DG', degree)
V = firedrake.VectorFunctionSpace(mesh, 'DG', degree)
Z = Q * V * Q * Q

These temperature and salinity values are what you might find around, say, the Filchner-Ronne Ice Shelf.
The "hssw" stands for *high-salinity shelf water*.
This terminology is confusing because by "shelf" they mean continental shelf, not ice shelf.

In [ ]:
from firedrake import Constant
T_hssw = Constant(-1.91)
s_hssw = Constant(34.65)

Melted ice shelf water (ISW) is very cold but has virtually no salt and so is more buoyant.

In [ ]:
T_isw = Constant(-3.5)
s_isw = Constant(0.0)

In [ ]:
a = Constant(0.05)
T = Constant((1 - a) * T_hssw + a * T_isw)
s = Constant((1 - a) * s_hssw + a * s_isw)

The factor that we're writing here as $\delta\ln\rho$ is the relative density contrast of the plume water over the ambient ocean.

In [ ]:
from plumes import coefficients
β_T = coefficients.thermal_expansion
β_S = coefficients.haline_contraction

δlnρ_0 = β_S * (s_hssw - s) - β_T * (T_hssw - T)
print(float(δlnρ_0))

In [ ]:
x = firedrake.SpatialCoordinate(mesh)

In [ ]:
z_in = Constant(-600.0)
z_out = Constant(-300.0)
X = x[0] / Lx
z_d = (1 - X) * z_in + X * z_out

In [ ]:
D_in = Constant(0.1)
u_in = Constant(0.01)

In [ ]:
δD = Constant(0.0)
δu = Constant(0.0)

The plume model is more complex than the advection and shallow water models.
It includes lots of physical processes besides just the transport of water.
You could try to spin up a model including all of these processes from a cold initial state, but you're probably going to have a bad time.
Instead, we've encapsulated all of these processes into a class called `PlumePhysics`.
You can override the member functions of this class in order to simplify the model and that's what we'll do to spin it up to steady state.
First, let's look at what member functions `PlumePhysics` contains.

In [ ]:
from plumes.models import plume
help(plume.PlumePhysics)

So the `PlumePhysics` class has four member functions:

* `melting_temperature`: What is the melting point of ice?
This depends on the salinity and pressure.
We can calculate the pressure based on the density of seawater and the ice shelf draft.
* `melt_rate`: Knowing the melting point, how fast is the ice melting?
The melt rate depends on how far the plume temperature is above the melting temperature and how fast the plume can remove the latent heat.
* `entrainment_rate`: How fast is the plume pulling in water from the ambient ocean? This is proportional to the correlation between the plume velocity and the slope of the ice shelf.
* `density_contrast`: How buoyant is the plume water relative to the ambient ocean?
This is the the $\delta\ln\rho$ term we've written above and it acts like a multiplicative factor before the gravitational constant.
This term then also modifies the shallow water wave speed.

To start with a more simplified model, we'll create a subclass of `PlumePhysics` and override three of these methods in order to turn off a few feedbacks at first.
First, we'll turn off all feedbacks to the melting temperature, which we'll set to be 0${}^\circ$C.
We'll also set the melt rate to 0; the plume will grow only by entrainment of the background ocean.
Finally, we'll set the relative density to be a constant, turning off the feedback of temperature and salinity into this variable.

In [ ]:
class SimplePhysics(plume.PlumePhysics):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.temperature_switch = firedrake.Constant(0.0)
        self.melt_rate_switch = firedrake.Constant(0.0)
        self.density_contrast_switch = firedrake.Constant(0.0)
        self.entrainment_switch = firedrake.Constant(0.0)
        
    def melting_temperature(self, fields, **inputs):
        T_m = super().melting_temperature(fields, **inputs)
        α = self.temperature_switch
        return (1 - α) * firedrake.Constant(0.0) + α * T_m
        
    def melt_rate(self, fields, **inputs):
        m = super().melt_rate(fields, **inputs)
        α = self.melt_rate_switch
        return (1 - α) * firedrake.Constant(0.0) + α * m

    def density_contrast(self, fields, **inputs):
        δlnρ = super().density_contrast(fields, **inputs)
        α = self.density_contrast_switch
        return (1 - α) * firedrake.Constant(δlnρ_0) + α * δlnρ
    
    def entrainment_rate(self, fields, **inputs):
        e = super().entrainment_rate(fields, **inputs)
        α = self.entrainment_switch
        return (1 - α) * firedrake.Constant(0.0) + α * e
    
physics = SimplePhysics()

The entrainment rate function will be the same as in the parent class.
Now just to tidy things up a bit we'll create some dictionaries that describe the external inputs to the system -- the state of the ambient ocean and the ice shelf cavity geometry -- as well as the forcing at the system boundary.

In [ ]:
from firedrake import as_vector

inputs = {
    'temperature_ambient': T,
    'salinity_ambient': s,
    'ice_shelf_draft': z_d
}

bcs = {
    'thickness_in': D_in,
    'momentum_in': D_in * Constant((u_in, 0.0)),
    'energy_in': Constant(D_in * T),
    'salt_in': Constant(D_in * s),
    'inflow_ids': (1,),
    'outflow_ids': (2,)
}

Now we'll create the equations describing the plume model with the input data we've created above.
The transport equation includes all terms describing the fluxes of mass, momentum, heat, and salt.
The source equation includes sources of mass, heat, and salt through melting of the ice shelf and entrainment of the ambient ocean.
These equations are non-stiff, so these will get lumped together and passed as the explicit part of an IMEX scheme.

The plume model defines the sink equation -- which includes frictional dissipation of momentum and relaxation of the plume temperature to the freezing point -- as a separate function because these terms are stiff.
We'll pass this equation as the implicit part of the IMEX scheme.

In [ ]:
def equation(fields):
    transport_equation = plume.make_transport_equation(
        physics, **inputs, **bcs
    )

    source_equation = plume.make_source_equation(
        physics, **inputs, **bcs
    )
    
    return transport_equation(fields) + source_equation(fields)


sink_equation = plume.make_sink_equation(
    physics, **inputs, **bcs
)

First we have to find a timestep that satisfies the CFL condition.

In [ ]:
import numpy as np
from plumes.coefficients import gravity as g
C_0 = abs(float(u_in)) + np.sqrt(float(δlnρ_0) * g * float(D_in + δD))
δx = mesh.cell_sizes.dat.data_ro[:].min()
timestep = (δx / 20) / C_0 / (2 * degree + 1)

final_time = 5 * Lx / C_0
num_steps = int(final_time / timestep)
dt = final_time / num_steps

In [ ]:
print(f'Final time:     {final_time / (24 * 60 * 60):5.3f} days')
print(f'Wave speed:     {C_0:5.3f} m / s')
print(f'Min cell size:  {δx:5.3f} m')
print(f'Timestep:       {timestep:5.3f} s')

In [ ]:
fields = firedrake.Function(Z)
D, q, E, S = fields.split()

D.project(D_in + δD * X)
q.project(D * as_vector((u_in + δu * X, 0.0)))
E.project(D * T)
S.project(D * s)

parameters = {
    'form_compiler_parameters': {
        'quadrature_degree': 4
    }
}

from plumes import numerics
integrator = numerics.IMEX(
    equation,
    sink_equation,
    fields,
    timestep,
    **parameters
)

In order to guarantee numerical stability, we'll have to keep track of what the maximum wave speed is throughout the domain at every timestep.
We'll then adjust the timestep down if need be.

In [ ]:
import tqdm
from firedrake import sqrt, inner, max_value
Q_0 = firedrake.FunctionSpace(mesh, family='DG', degree=0)
wave_speed = firedrake.Function(Q_0)

limiter = firedrake.VertexBasedLimiter(Q)
elapsed_time = 0.0

with tqdm.tqdm(total=final_time, unit_scale=True) as progress_bar:
    while elapsed_time < final_time:
        D, q = integrator.state.split()[:2]
        u = q / D

        wave_speed.project(sqrt(inner(u, u)) + δlnρ_0 * g * D)
        C = wave_speed.dat.data_ro[:].max()
        cfl_timestep = δx / C / (2 * degree + 1)
        dt = min(dt, cfl_timestep / 4)

        Dmin, Dmax = D.dat.data_ro[:].min(), D.dat.data_ro[:].max()
        progress_bar.set_description(f'{Dmin:5.3f}, {Dmax:5.3f}')

        integrator.step(dt)
        limiter.apply(D)

        elapsed_time += dt
        physics.entrainment_switch.assign(
            max(0, min(1, (elapsed_time - Lx / C_0) / (3 * final_time / 4)))
        )
        progress_bar.update(dt)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(D, axes=axes)
fig.colorbar(colors)